# YoloV5 Training at CPU Example

Inferência no conjunto de treino ou teste do UAVDT dataset utilizando pesos pré-treinados no COCO Dataset.

Autor: Yuri Gonçalves Ribeiro

Data: 15 de Agosto de 2020

Dataset Utilizado: UAV Dataset

Referências:

https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

https://towardsdatascience.com/how-to-train-a-custom-object-detection-model-with-yolo-v5-917e9ce13208

https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

**É necessário que se ative o environment correto para esse notebook funcionar.**

Exemplo 1:
```bash
conda activate yv5
jupyter notebook
```

Exemplo 2:
```bash
conda activate yv5; experimento="YOLOv5_UAVDT_1_TRAIN"; outputFileName=$(date "+"$experimento"_ipynb_%d_%B_%Y_%Hh_%Mm_%Ss"); echo "Salvando log de execução em: $outputFileName.txt"; nohup jupyter nbconvert --to notebook --execute --allow-errors --output "./logs/$outputFileName" --ExecutePreprocessor.timeout=-1 --Application.log_level=10 $experimento.ipynb &> "./logs/$outputFileName.txt" &
```

# Etiqueta do Experimento (Yolov5 Commit Hash, Conda Env, Informações da Máquina)

###### Commit Hash da Rede YoloV5 utilizada neste experimento e pacotes instalados no conda environment

In [ ]:
%%script bash
echo "Hash key do commit YoloV5 que esta em uso:"
cd ../../Submodules/yolov5
echo $( git rev-parse --verify HEAD)
echo
echo "Verificar localizacao do python sendo executado:"
which python
echo
echo "Conda Environment Utilizado: "
conda env export

###### Especificações do Computador e do Experimento

In [ ]:
# Especificações do computador, para saber onde estou rodando o código...
import datetime
from platform import uname
from os import environ

start_time = datetime.datetime.now()
time_stamp = start_time.strftime("%d_%B_%Y_%Hh_%Mm_%Ss")

username = environ['USER']
sysinfo = uname()
hostname = sysinfo.node

root = "~/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_1"

dropbox_dir = "~/Dropbox"

experimento = "YOLOv5_UAVDT_1"

output_dir = "%s/YoloV5_Runs/Experimentos/%s_%s"%(dropbox_dir,experimento,time_stamp)


# Checar conda environment
conda_env = !printenv CONDA_PROMPT_MODIFIER
if conda_env != ['(yv5) ']:
    raise SystemExit("ERRO.. Conda environment incorreto.")

In [ ]:
print("""
==Computer Specifications==
username: {username}

hostname: {hostname}

system info: {sysinfo}

{{root}}: {root}

dropbox dir: {dropbox_dir}

experimento: {experimento}

time stamp: {time_stamp}

output dir: {output_dir}
===========================
""".format(username = username, hostname = hostname, sysinfo = sysinfo,
           conda_env = conda_env, root = root, dropbox_dir = dropbox_dir,
           time_stamp = time_stamp, experimento = experimento, output_dir = output_dir))

In [ ]:
!nvidia-smi

# Clonar o Repositório YOLOv5

In [ ]:
#Ir para a raiz
%cd {root}
!echo
#Criar pasta de logs e runs
!mkdir logs runs
!echo
#Remover diretórios antigos do yolov5
!rm -rfv yolov5
!echo
#Copiar diretório novo da pasta submodules
!cp -av ../../Submodules/yolov5/ yolov5/
!echo
!ls

## Entrar na pasta yolov5 e instalar os requirements.txt

In [ ]:
%cd {root}/yolov5/
%pip install -U -r requirements.txt

# Executar Script para Transformar Imagens do UAVDT Dataset para o formato requerido pelo YOLOv5

In [ ]:
# Ir para a raiz
%cd {root}

In [ ]:
%%script bash
# Apagar os links existentes para o dataset
rm -rfv UAVDT_YOLOv5
# Criar novos hard links para o dataset e organizer o dataset no formato exigido pelo yolov5
mkdir UAVDT_YOLOv5
# Executar o script para converter UAVDT para o formato YOLOv5:
cd ../../misc

python uavdt_to_yolov5.py --conjunto treinoEteste --dest ../RedesNeurais/YOLOv5_UAVDT_1/UAVDT_YOLOv5
#python uavdt_to_yolov5.py --videos M0101 M0203 --dest ../RedesNeurais/YOLOv5_UAVDT_0/UAVDT_YOLOv5

cd ../RedesNeurais/YOLOv5_UAVDT_1/

# Criar um data.yaml conforme exigido pelo yolov5


In [ ]:
# O conteúdo original não está adequado... mais fácil que editar: removerei-o e criarei um novo.

datayaml = """# train and val datasets (image directory or *.txt file with image paths)

train: ../UAVDT_YOLOv5/train/images/
val: ../UAVDT_YOLOv5/test/images/

# number of classes
nc: 3

# class names
names: ['car', 'truck', 'bus']
"""

%rm data.yaml
with open('data.yaml', 'w') as file:
    file.write(datayaml)
%cat data.yaml

# Executar Treinamento por 100 Eṕocas

In [ ]:
%cd {root}/yolov5

!python train.py --epochs 300 --cfg ./models/yolov5l.yaml --weights ../../../Weights/YoloV5/yolov5l.pt  --data ../data.yaml --img 640 --batch-size 16 --device 1 --logdir ../runs --workers 4

# Salvar Resultados no Dropbox
Nesta etapa, salvarei os arquivos importantes no dropbox.

In [ ]:
%cd {root}
%mkdir -p {output_dir}
%cp -av logs/ {output_dir}
%cp -av runs/ {output_dir}

# Finalização

In [ ]:
finish_time = datetime.datetime.now()
delta_time = finish_time - start_time
print("""
Start: {start}

Finish: {finish}

Duration (hour:min:sec.microsec):  {delta}
""".format(start = start_time, finish = finish_time, delta = delta_time))